In [36]:
class AttrDisplay:
    """
    提供一个可继承的显示重载方法，该方法显示具有类名的实例，
    以及存储在实例本身上的每个属性的名称=值对（但不包括从其类继承的属性）。
    这个类可以混合到任何类中，并且可以在任何实例上工作。
    """
    def gatherAttrs(self):
        attrs = []
        for key in sorted(self.__dict__):
            attrs.append(f'{key}={getattr(self, key)}')
            # getattr(self, key)：动态获取实例 self 中名为 key 的属性值。等价于 self.key，但更灵活（key 是字符串变量）。
        return ', '.join(attrs)
    def __str__(self):
        return f'[{self.__class__.__name__}: {self.gatherAttrs()}]'

In [37]:
class Person(AttrDisplay):
    """
    创建和处理人员记录
    """
    def __init__(self, name, job=None, pay=0):
        self.name = name
        self.job = job
        self.pay = pay
    def lastName(self): # Assumes last is last
        return self.name.split()[-1]
    def giveRaise(self, percent): # Percent must be 0..1
        self.pay = int(self.pay * (1 + percent))
        
class Manager(Person):
    """
    具有特殊要求的定制人员
    """
    def __init__(self, name, pay):
        Person.__init__(self, name, 'mgr', pay) # 工作名称是隐含的manager
    def giveRaise(self, percent, bonus=.10):
        Person.giveRaise(self, percent + bonus)

super().\_\_init\_\_()
* 自动查找父类 并调用其 __init__ 方法。适用于 单继承和多继承（遵循方法解析顺序 MRO）
* 无需显式指定父类名（减少硬编码）。在多继承中能正确处理菱形继承（Diamond Inheritance）问题。推荐用法（符合现代 Python 风格）。

In [38]:
class Manager(Person):
    def __init__(self, name, pay):
        super().__init__(name, 'mgr', pay)  # Python 3 简写
        # 或 super(Manager, self).__init__(name, 'mgr', pay)  # Python 2/3 通用

Person.\_\_init\_\_(self, name, 'mgr', pay)
* 显式指定父类 Person 并调用其 __init__ 方法。直接操作，不依赖 MRO 机制。
* 代码意图更明确（直接看到调用的父类）。在多继承中可能引发问题（如父类被重复初始化）。适用于需要 强制指定某个父类 的场景。

In [39]:
class Manager(Person):
    def __init__(self, name, pay):
        Person.__init__(self, name, 'mgr', pay)  # 显式调用父类

\_\_name\_\_是 Python 的一个内置变量，表示当前模块的名称：
* 如果模块是直接运行的，\_\_name\_\_ 的值是 '\_\_main\_\_'。
* 如果模块是被其他文件导入的，\_\_name\_\_ 的值是模块的文件名（不含 .py 后缀）。
  
if \_\_name\_\_ == '\_\_main\_\_':
* 当条件成立时，说明当前模块是主程序入口，会执行下方的代码块。
* 如果模块被导入，此条件不成立，下方的代码块不会执行. 

In [40]:
if __name__ == '__main__':
    bob = Person('Bob Smith')
    sue = Person('Sue Jones', job='dev', pay=100000)
    print(bob)
    print(sue)
    print(bob.lastName(), sue.lastName())
    sue.giveRaise(.10)
    print(sue)
    tom = Manager('Tom Jones', 50000)
    tom.giveRaise(.10)
    print(tom.lastName())
    print(tom)

[Person: job=None, name=Bob Smith, pay=0]
[Person: job=dev, name=Sue Jones, pay=100000]
Smith Jones
[Person: job=dev, name=Sue Jones, pay=110000]
Jones
[Manager: job=mgr, name=Tom Jones, pay=55000]


---
#### 把对象存储到数据库
pickle 和 shelve
* pickle模块可以将Python对象序列化为二进制格式（或字符串），以便存储到文件或网络传输。并且在反序列化时恢复为原始对象（包括自定义类的实例）。
* shelve模块基于 pickle 实现，提供类似字典的持久化存储（键值对形式）。数据存储在磁盘（通常是一个或多个文件），键必须是字符串，值可以是任意 pickle 支持的 Python 对象
* shelve 和常规字典之间的唯一区别是：一开始必须打开 shelve 并且在修改之后必须关闭它。
* 永远避免用 pickle 反序列化不受信任的数据（如网络传输）。因为pickle在反序列化时可能执行任意代码（永远不要反序列化不受信任的数据）。而且只能被 Python 读取，无法与其他语言交互。以及不同 Python 版本的 pickle 格式可能不兼容。

优先学习现代工具：
* 数据交换：JSON、msgpack、Protocol Buffers。
* 数据库：SQLite、Redis、TinyDB。

1. 在 shelve 数据库中存储对象:

In [47]:
bob = Person('Bob Smith')
sue = Person('Sue Jones', job='dev', pay=100000)
tom = Manager('Tom Jones', 50000)

import shelve

# 存储对象的文件名
db = shelve.open('persondb') # shelve.open() 打开一个持久化的字典（shelve 数据库）, 如果 persondb 文件不存在，shelve.open('persondb') 会自动创建它
for obj in (bob, sue, tom): # 使用对象的名称attr作为键
    db[obj.name] = obj      # 按键在shelve上存储对象
# 更改后关闭
db.close()

In [48]:
# 文件名模式匹配库g lob.glob('persondb*') 是一种快速查找 shelve 数据库文件的方法。
import glob
glob.glob('persondb*') # 返回所有文件名以 'persondb' 开头的文件列表。
# 这里的 * 是通配符，表示匹配任意数量的任意字符（包括空字符）。
# 例如：'persondb*' 可以匹配 'persondb'、'persondb.db'、'persondb.dat' 等。

['persondb', 'persondb-shm', 'persondb-wal', 'persondb.dir']

In [53]:
with shelve.open('persondb') as db:
    # 转换为字典查看
    print(dict(db))
db.close()

{'Bob Smith': <__main__.Person object at 0x0000028558749470>, 'Sue Jones': <__main__.Person object at 0x00000285587491D0>, 'Tom Jones': <__main__.Manager object at 0x00000285588449E0>}


In [35]:
print(open('persondb', 'rb').read()) # 'rb'以二进制格式读取

b"SQLite format 3\x00\x10\x00\x02\x02\x00@  \x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00.\x86*\r\x0f\xf8\x00\x02\x0fk\x00\x0fk\x0f\xcf\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [65]:
# Reopen the shelve
db = shelve.open('persondb')
print(len(db))

3


In [59]:
list(db.keys())     # keys is the index

['Bob Smith', 'Sue Jones', 'Tom Jones']

In [64]:
bob = db['Bob Smith'] 
print(bob)
print(type(bob))

[Person: job=None, name=Bob Smith, pay=0]
<class '__main__.Person'>


In [61]:
bob.lastName()

'Smith'

In [66]:
for key in db:     
    print(key, '=>', db[key])
db.close()

Bob Smith => [Person: job=None, name=Bob Smith, pay=0]
Sue Jones => [Person: job=dev, name=Sue Jones, pay=100000]
Tom Jones => [Manager: job=mgr, name=Tom Jones, pay=50000]


2. 更新 shelve 中的对象

In [68]:
# File updatedb.py：更新数据库中的Person对象

import shelve
# 重新打开具有相同文件名的shelve
db = shelve.open('persondb')

for key in sorted(db):
    print(key, '\t=>', db[key])

sue = db['Sue Jones']
sue.giveRaise(.10) # 使用类的方法在更新类
db['Sue Jones'] = sue # 分配到要在 shelve 上更新的键
db.close()

Bob Smith 	=> [Person: job=None, name=Bob Smith, pay=0]
Sue Jones 	=> [Person: job=dev, name=Sue Jones, pay=110000]
Tom Jones 	=> [Manager: job=mgr, name=Tom Jones, pay=50000]


一个小练习:
实现学生选课系统

学生类\
属性：姓名、学号、电话、所选课程列表\
方法：查看：显示该学生所有课程信息；添加课程：将选好的课程添加到课程列表中

课程类\
属性：课程编号、课程名称、教师名\
方法：查看：显示该课程的全部信息；设置教师：给当前课程安排一个教师

教师类：\
属性：教师编号，教师名、电话、所教课程列表\
方法：查看：查看该教师的所有课程\

完成以上三个类，并创建20名学生，6个课程，3名教师。给课程随机安排任课教师并给20名学生随机分配3个课程，最终显示这20名学生选课情况。